In [1]:
# coding: utf-8
import requests
import csv
from bs4 import BeautifulSoup

def get_the_page(url): #get one page's restaurants info
    
    r = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'})
    html_str = r.text
    mypage = BeautifulSoup(html_str,"html.parser")
    
    myitems = mypage.find_all('li',attrs={'class':'sr1-listing-content-cell pois-restaurant-list-cell '})
    
    data=[]
    
    for myitem in myitems:
        
        title = myitem.find('h2').text.strip()  #find titles
             
        like=myitem.find('span',attrs={'class':'score score-big highlight'})  #find likes        
        if like is not None:       #some item doesn't have like or other features
            like = like.text.strip()
        else:
            like = None
            
        location = myitem.find('div',attrs={'class':'icon-info address'}) #find locations
        if location is not None:      
            location = location.find('span')
            if location is not None:
                location = location.text.strip()
        else:
            location = None
        
        price = myitem.find('div',attrs={'class': 'icon-info icon-info-food-price'}) #find prices
        if price is not None:       
            price = price.find('span').text.strip()
        else:
            price = None
        
        review = myitem.find('div',attrs={'class':'counters-container'})  #dind review numbers
        if review is not None:       
            review = review.find('span')
            if review is not None:
                review = review.text.strip()
        else:
            review = None
            
        bookmark = myitem.find('div',attrs={'class':'text bookmarkedUserCount js-bookmark-count'}) #find bookmark
        if bookmark is not None:       
            bookmark = bookmark['data-count']
        else:
            bookmark = None
            
        discount = myitem.find('div',attrs={'class':'info-row'}) #find discount info
        if discount is not None:       
            discount = discount.find('span')
            if discount is not None:
                discount = discount.text.strip()
        else:
            discount = None
            
        countries_material=myitem.find("ul",attrs={'class':'pois-categoryui-list'}).find_all('li')
        
        country = countries_material[0].text.strip()     #find country
        if len(countries_material)>1:
            style = countries_material[1].text.strip()    #find likes
        else:
            style = countries_material[0].text.strip()
                         
        data.append([title,location,price,country,style,like,review,bookmark,discount])
    
    print(url)

    with open('openrice_sample.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return

In [2]:
url_list = ['https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Lunch',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Tea',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Dinner']

In [3]:
def scrape_pages(url_page1):    #get 17 pages, which is limit of open rice, of the results return from your search
    get_the_page(url_page1)
    for q in range(1,17):
        page_number=str(q+1)
        url_next='{0}&page={1}'.format(url_page1,page_number)
        get_the_page(url_next)
    return

In [4]:
fail = []
for url in url_list:
    try: 
        scrape_pages(url)
    except Exception as e:
        print(e)
        print(url)
        fail.append((url, e))

https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=2
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=3
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=4
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=5
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=6
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=7
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=8
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-10-03&BookingPeriod=Breakfast&page=9
https://www.openr

In [5]:
import pandas
df = pandas.read_csv('openrice_sample.csv')

In [6]:
len(df)

931

In [7]:
df = df.drop_duplicates() #remove duplicated rows

In [8]:
len(df)

495

In [9]:
df

,LAB EAT Restaurant & Bar,尖沙咀金巴利道87-89號僑豐大厦地下1-2號舖,$201-400,西式,海鮮,436,(565 食評),50744,網上訂座可享75折優惠
0,Shine,尖沙咀北京道12A號太子集團中心6樓,$201-400,西式,酒,693.0,(777 食評),21136,送 25里數 / 30積分
1,Yadllie Plate,旺角西洋菜街1號兆萬中心11樓,$101-200,韓國菜,韓式炸雞,487.0,(630 食評),27681,早鳥及消夜時段訂座 可享全單八折優惠
2,漁獲浜燒 Toretore Hamayaki,銅鑼灣軒尼詩道525號澳門逸園中心18樓,$201-400,日本菜,日本菜,503.0,(524 食評),19647,送 25里數 / 30積分
3,心之食堂 Love Cafe,銅鑼灣耀華街3號百樂中心12樓1203室,$201-400,日本菜,壽司/刺身,490.0,(545 食評),6580,全單85折
4,The Grill Room,銅鑼灣駱克道459-461號The L. Square 5樓,$201-400,西式,扒房,508.0,(662 食評),32342,網上訂座八折優惠-商務二人或四人套餐
5,Sky726,旺角彌敦道724-726號25樓,$201-400,法國菜,甜品/糖水,369.0,(496 食評),38563,自選餐牌主菜買一送一優惠 或 甜品8折優惠； 飲品一律8折
6,Burgeroom,尖沙咀彌敦道132號美麗華廣場一期2 樓 207 號舖,$51-100,美國菜,漢堡包,437.0,(511 食評),13841,送 25里數 / 30積分
7,樂園 Happyland,尖沙咀北京道65-69號環球商業大廈2樓205-06室,$101-200,西式,樓上cafe,574.0,(637 食評),10097,送 25里數 / 30積分
8,The Hwaduk,銅鑼灣開平道1號Cubus 21樓,$101-200,韓國菜,韓式炸雞,450.0,(498 食評),16542,送 25里數 / 30積分
9,The Joomak,尖沙咀廣東道3-27號海港城海運大廈3樓LCX 34號舖,$101-200,韓國菜,沙律,470.0,(629 食評),21527,送 25里數 / 30積分


In [10]:
df.to_csv('sample.csv', encoding='utf-8', index=False)